In [1]:
from pymongo import MongoClient
import statistics

In [2]:
client = MongoClient()
db = client["Projet_Bachelor"]

In [4]:
# Âge moyen
under_40 = {"$match": {"age": {"$gt": 0, "$lt": 45}}}
avg_age = {"$group": {"_id": "", "moyenne": {"$avg": "$age"}}}
list(db.players.aggregate([under_40, avg_age]))

[{'_id': '', 'moyenne': 26.3357772713762}]

In [5]:
# Âge maximum des joueurs (sans prendre les valeurs extrêmes):
under_40 = {"$match": {"age": {"$gt": 0, "$lt": 45}}}
max_age = {"$group": {"_id": "", "max": {"$max": "$age"}}}
max_age_under_40 = db.players.aggregate([under_40, max_age])

list(max_age_under_40)

[{'_id': '', 'max': 44}]

In [6]:
#  Âge minimum des joueurs (sans prendre les valeurs extrêmes):
under_40 = {"$match": {"age": {"$gt": 0, "$lt": 45}}}
min_age = {"$group": {"_id": "", "min": {"$min": "$age"}}}
list(db.players.aggregate([under_40, min_age]))

[{'_id': '', 'min': 17}]

In [7]:
group_position = {"$group":
                          {"_id": "$position",
                           "count": {"$sum": 1}
                           }}
list(db.players.aggregate([group_position]))

[{'_id': 'RM', 'count': 1104},
 {'_id': 'CM', 'count': 2328},
 {'_id': 'LM', 'count': 1137},
 {'_id': 'RWB', 'count': 197},
 {'_id': 'LB', 'count': 1364},
 {'_id': 'RB', 'count': 1380},
 {'_id': 'LF', 'count': 10},
 {'_id': 'CAM', 'count': 1239},
 {'_id': 'GK', 'count': 2128},
 {'_id': 'CDM', 'count': 1694},
 {'_id': 'LWB', 'count': 195},
 {'_id': 'RF', 'count': 6},
 {'_id': 'LW', 'count': 503},
 {'_id': 'CF', 'count': 217},
 {'_id': 'CB', 'count': 3490},
 {'_id': 'RW', 'count': 563},
 {'_id': 'ST', 'count': 3031}]

In [9]:
#  Moyenne de la note générale:
true_mean = {"$group":
                 {"_id": "",
                  "given_mean": {"$avg": "$rating"}
                  }
             }

list(db.players.aggregate([true_mean]))

[{'_id': '', 'given_mean': 68.01680753910425}]

In [10]:
# Récupération des 100 meilleurs joueurs:
sort = {'$sort': {'uniqueCount': -1}}
limitation = {'$limit': 100}
unique = {"$group": {"_id": "$name", "uniqueCount": {"$max": "$rating"}}}
list(db.players.aggregate([unique, sort, limitation]))

[{'_id': 'Pelé', 'uniqueCount': 99},
 {'_id': 'Lionel Messi', 'uniqueCount': 98},
 {'_id': 'Robert Lewandowski', 'uniqueCount': 98},
 {'_id': 'Diego Maradona', 'uniqueCount': 98},
 {'_id': 'Rúben Dias', 'uniqueCount': 97},
 {'_id': 'Cristiano Ronaldo', 'uniqueCount': 97},
 {'_id': 'Luiz Frello Filho Jorge', 'uniqueCount': 97},
 {'_id': 'Zinedine Zidane', 'uniqueCount': 97},
 {'_id': 'Kylian Mbappé', 'uniqueCount': 97},
 {'_id': 'Ronaldo', 'uniqueCount': 97},
 {'_id': "N'Golo Kanté", 'uniqueCount': 96},
 {'_id': 'Gianluigi Donnarumma', 'uniqueCount': 96},
 {'_id': 'Kevin De Bruyne', 'uniqueCount': 96},
 {'_id': 'Lev Yashin', 'uniqueCount': 95},
 {'_id': 'Johan Cruyff', 'uniqueCount': 95},
 {'_id': 'Sadio Mané', 'uniqueCount': 95},
 {'_id': 'Ferenc Puskás', 'uniqueCount': 95},
 {'_id': 'Ronaldinho', 'uniqueCount': 95},
 {'_id': 'Marquinhos', 'uniqueCount': 95},
 {'_id': 'Paolo Maldini', 'uniqueCount': 95},
 {'_id': 'Garrincha', 'uniqueCount': 95},
 {'_id': 'Marcus Rashford', 'uniqueCount

In [11]:
# Récupération des 11 meilleurs joueurs de fifa selon leur notes attribuées et selon leur moyenne aggrégée (en changeant seulement le sort):
sort = {'$sort': {'rating': -1}}
position_wanted = ["GK", "CB", "RB", "LB", "CM", "CAM", "RW", "LW", "ST"]
projection = {"$project": {
    "_id": 0,
    "name": 1,
    "rating": 1,
    "moyenne aggrégée": {"$divide": [{"$add": ["$physicality", "$passing", "$dribbling", "$pace", "$shooting", "$defending"]}, 6]},
    }
}

for position in position_wanted:
    if position in ["CB", "CM"]:
        limitation = {'$limit': 2}
    else:
        limitation = {'$limit': 1}
    find_post = {"$match": {"position": position}}
    players = list(db.players.aggregate([find_post, projection, sort, limitation]))
    for player in players:
        print(f"{position}: {player['name']} {player['rating']} {player['moyenne aggrégée']}")


GK: Gianluigi Donnarumma 96 0.0
CB: Rúben Dias 97 82.5
CB: Paolo Maldini 95 80.33333333333333
RB: Carlos Alberto 94 87.16666666666667
LB: Marcelo Vieira da Silva 94 91.33333333333333
CM: Luiz Frello Filho Jorge 97 90.33333333333333
CM: Kevin De Bruyne 96 89.5
CAM: Pelé 99 87.83333333333333
RW: Lionel Messi 98 83.83333333333333
LW: Ronaldinho 95 83.0
ST: Robert Lewandowski 98 86.33333333333333


In [12]:
print("\nAGGREGATED TEAM\n")

sort = {"$sort": {'moyenne aggrégée': -1}}
for position in position_wanted:
    if position in ["CB", "CM"]:
        limitation = {'$limit': 2}
    else:
        limitation = {'$limit': 1}
    find_post = {"$match": {"position": position}}
    players = list(db.players.aggregate([find_post, projection, sort, limitation]))
    for player in players:
        print(f"{position}: {player['name']} {player['rating']} {player['moyenne aggrégée']}")


AGGREGATED TEAM

GK: Jonathan Orozco 75 44.0
CB: Ronald Koeman 93 87.83333333333333
CB: Marquinhos 95 84.83333333333333
RB: Achraf Hakimi 93 89.66666666666667
LB: Marcelo Vieira da Silva 94 91.33333333333333
CM: Luiz Frello Filho Jorge 97 90.33333333333333
CM: Kevin De Bruyne 96 89.5
CAM: Zinedine Zidane 97 89.83333333333333
RW: Mohamed Salah 94 84.66666666666667
LW: Diogo Jota 88 83.83333333333333
ST: Tim Cahill 91 87.83333333333333


In [14]:
num_multi = "MOYENNE"
true_mean = {"$group":
                     {"_id": "",
                      "given_mean": {"$avg": "$rating"}
                      }
                 }
mean_ratings = {"$group":
                    {"_id": "",
                     "mean_rating": {"$avg": "$rating"},
                     "mean_pace": {"$avg": "$pace"},
                     "mean_shooting": {"$avg": "$shooting"},
                     "mean_passing": {"$avg": "$passing"},
                     "mean_dribbling": {"$avg": "$dribbling"},
                     "mean_defending": {"$avg": "$defending"},
                     "mean_physicality": {"$avg": "$physicality"},
                     }
                }
attributes = ['Rating', 'Pace', 'Shooting', 'Passing', 'Dribbling', 'Defending', 'Physicality']
if num_multi == 'MOYENNE':
    ratings = list(
        db.players.find({}, {"pace": 1, "shooting": 1, "passing": 1, "dribbling": 1, "defending": 1,
                                    "physicality": 1, '_id': 0}))
    given_moy = int(list(db.players.aggregate([true_mean]))[0]['given_mean'])
    ratings_list = [values for key, values in list(db.players.aggregate([mean_ratings]))[0].items()][
                   1:]
else:
    ratings = list(
        db.players.find({"name": num_multi}, {"pace": 1, "shooting": 1, "passing": 1, "dribbling": 1, "defending": 1,
                                    "physicality": 1, '_id': 0}))
    match_player = {"$match": {"name": str(num_multi)}}
    given_moy = int(list(db.players.aggregate([match_player, true_mean]))[0]['given_mean'])
    ratings_list = [values for key, values in list(db.players.aggregate([match_player, mean_ratings]))[0].items()][
                   1:]
ratings_values = []
for items in ratings:
    for key in items:
        ratings_values.append(items[key])
ratings_moy = int(statistics.mean(ratings_values))
print(num_multi)
print(f"Moyenne attribuée : {given_moy}")
print(f"Moyenne calculée selon les attributs : {ratings_moy}")

MOYENNE
Moyenne attribuée : 68
Moyenne calculée selon les attributs : 57
